# Step: Read Documents - autoloader

**Pipeline**: autoloader  
**Step**: read_documents  
**Job Template**: Auto Loader

**Purpose**: Read documents using Auto Loader and store in Bronze Delta table

**Inputs**:
- Source: Configuration-driven from `autoloader_config.yaml`

**Outputs**:
- Bronze table: `${catalog}.${bronze_schema}.autoloader_documents_bronze`

In [ ]:
# Import required libraries
import json
from pyspark.sql import functions as F
from databricks import dbutils

In [ ]:
# Load configuration from task variables (set by setup task)
config_json = dbutils.jobs.taskValues.get(taskKey="setup", key="config")
config = json.loads(config_json)

# Get step configuration
pipeline_config = config["pipelines"]["autoloader"]
step_config = pipeline_config["pdf"]["standard"]["steps"][0]  # First step: read_documents

service_name = step_config["service"]
service_config = config["services"][service_name]

source_path = step_config["source"]
output_table = step_config["output"]

print(f"Source: {source_path}")
print(f"Output: {output_table}")

In [ ]:
# Read using Auto Loader (streaming)
df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", service_config["format"])
    .option("cloudFiles.schemaLocation", service_config["schema_location"])
    .load(source_path)
)

# Add metadata
df_enriched = (
    df
    .withColumn("file_name", F.input_file_name())
    .withColumn("file_path", F.input_file_name())
    .withColumn("_ingestion_timestamp", F.current_timestamp())
    .withColumn("_processing_date", F.current_date())
)

# Write to Bronze table
(
    df_enriched.writeStream
    .format("delta")
    .option("checkpointLocation", service_config["checkpoint_location"])
    .trigger(availableNow=True)
    .toTable(output_table)
).awaitTermination()

print(f"✅ Documents written to {output_table}")